<a href="https://colab.research.google.com/github/tathan08/review-rater/blob/main/notebooks/00_colab_complete_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline Architecture 2.0 - Training Phase

## Data Flow Overview

```
data/raw → (external processing) → data/clean
data/clean → (00_ipynb + gemini) → data/pseudo-label  
data/pseudo-label → data/testing + data/training
data/clean → data/training (combined)
HuggingFace model trained on data/training with feedback loop against gemini
Trained models → models/saved_models
```

## Directory Structure

### Data Directories
- **`data/raw`**: Raw input data (processed externally)
- **`data/clean`**: Cleaned/processed data from data/raw
- **`data/pseudo-label`**: Pseudo-labeled data generated by Gemini from data/clean
- **`data/training`**: Training data (combination of data/clean + data/pseudo-label)
- **`data/testing`**: Testing data split from data/pseudo-label
- **`data/actual`**: Production data for inference (used by 01_inference_pipeline.ipynb)

### Model Directories
- **`models/saved_models`**: Trained models ready for production
- **`models/cache`**: Model cache files

### Results Directories
- **`results/predictions`**: Training predictions and evaluations
- **`results/inference`**: Production inference results

## Pipeline Components

### Training Phase (This Notebook - 00_ipynb)
1. **Environment Setup**: Install packages, configure GPU
2. **Data Processing**: Create directory structure, load sample data
3. **Gemini Pseudo-Labeling**: Generate high-quality labels for training
4. **HuggingFace Training**: Train models with feedback loop against Gemini
5. **Model Export**: Save trained models to models/saved_models

### Inference Phase (01_ipynb)
1. **Load Trained Models**: From models/saved_models
2. **Process Production Data**: From data/actual
3. **Generate Predictions**: Using trained pipeline
4. **Save Results**: To results/inference

## Integration Points
- **Spam Detection**: Pipeline ready for spam detection model integration
- **Feedback Loop**: HuggingFace model iteratively improved against Gemini predictions
- **Production Ready**: Complete separation of training and inference phases

# Review Classification Pipeline - Complete Google Colab Implementation

This notebook implements the complete review classification pipeline for detecting Google review policy violations, fully configured for Google Colab.

## Pipeline Overview

### Phase 1: Environment Setup and Data Structure
- Install all required packages (transformers, torch, google-generativeai, etc.)
- Create proper directory structure (data/clean, data/pseudo-label, etc.)
- Load sample data for demonstration

### Phase 2: Core Pipeline Components
- **Ollama Pipeline**: Local LLM classification (for reference, not runnable in Colab)
- **HuggingFace Pipeline**: Zero-shot classification using pre-trained models
- **Gemini Pseudo-Labeling**: High-quality label generation for training data
- **Ensemble Method**: Combines multiple approaches for best results

### Phase 3: Future Spam Detection Integration
- Pipeline output will be piped into a spam detection model
- Structured JSON output format for downstream processing
- Confidence scoring for reliable filtering

### Phase 4: Evaluation and Analysis
- Comprehensive performance metrics
- Policy category accuracy assessment
- Model comparison and improvement recommendations

**Key Features:**
- **Policy Categories**: No_Ads, Irrelevant, Rant_No_Visit detection
- **Zero Setup**: Everything configured for Google Colab
- **Extensible**: Ready for spam detection integration
- **Production Ready**: Structured output and comprehensive evaluation

## 1. Environment Setup

In [ ]:
# Install required packages for the complete pipeline
!pip install -q transformers==4.43.3 torch pandas scikit-learn
!pip install -q google-generativeai tqdm datasets accelerate
!pip install -q ipywidgets matplotlib seaborn wordcloud

print("✅ Core packages installed successfully!")

# Check GPU availability and setup device
import torch
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("Using CPU - models will run slower but still functional")

# Set environment for optimal performance
os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # Avoid warnings
print("Environment configured for optimal performance")

ERROR: Operation cancelled by user
^C


## 2. Project Structure Setup

In [ ]:
# Create complete directory structure matching the actual pipeline
import os
import pandas as pd
import json
from pathlib import Path

# Create all necessary directories (matching actual pipeline structure)
directories = [
    # Source code structure
    'src/config', 'src/core', 'src/pseudo_labelling', 'src/pipeline', 'src/integration',

    # Data directories (matching actual structure)
    'data/raw',           # For raw input data
    'data/clean',         # For cleaned/processed data (from data/raw)
    'data/pseudo-label',  # For pseudo-labeled data from Gemini (from data/clean)
    'data/training',      # For training data split (from data/clean + data/pseudo-label)
    'data/testing',       # For testing data split (from data/pseudo-label)
    'data/actual',        # For actual production data to be processed by 01_inference_pipeline.ipynb
    'data/sample',        # For sample data

    # Results directories
    'results/predictions',   # All predictions
    'results/evaluations',   # For evaluation results
    'results/reports',       # For generated reports

    # Other directories
    'models/saved_models',   # For trained models
    'models/cache',          # For model cache
    'logs/pipeline_logs',    # For pipeline logs
    'prompts',               # Prompt engineering
    'docs'                   # Documentation
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    # Create __init__.py files for Python packages
    if directory.startswith('src/'):
        with open(f'{directory}/__init__.py', 'w') as f:
            f.write('# Review Classification Pipeline Package\n')

print("✅ Complete directory structure created!")
print(f"Created {len(directories)} directories")

# Verify critical directories exist
critical_dirs = ['data/clean', 'data/pseudo-label', 'data/sample', 'results/predictions']
for dir_name in critical_dirs:
    if os.path.exists(dir_name):
        print(f"✅ {dir_name}")
    else:
        print(f"❌ {dir_name} - MISSING!")

print("\nDirectory structure matches production pipeline!")

## 3. Sample Data Creation

In [ ]:
# Load actual sample data from the production pipeline
sample_data = {
    'id': [1, 2, 3, 4, 5],
    'text': [
        "Use my promo code EAT10 for 10% off! DM me on WhatsApp.",
        "Great laksa; broth was rich and staff friendly. Will return.",
        "Crypto is the future. Buy BTC now! Nothing to do with this cafe.",
        "Overpriced scammers. Society is doomed.",
        "Visited on 18 Aug, ordered set A; cashier fixed a double-charge."
    ],
    'gold_label': ['REJECT', 'APPROVE', 'REJECT', 'REJECT', 'APPROVE'],
    'gold_category': ['No_Ads', 'None', 'Irrelevant', 'Rant_No_Visit', 'None']
}

df = pd.DataFrame(sample_data)
df.to_csv('data/sample/sample_reviews.csv', index=False)

print("✅ Production sample data loaded!")
print("\nSample Data Overview:")
print(df.to_string(index=False))

print(f"\nLabel Distribution:")
print(f"APPROVE: {len(df[df['gold_label'] == 'APPROVE'])} reviews")
print(f"REJECT:  {len(df[df['gold_label'] == 'REJECT'])} reviews")

print(f"\nCategory Distribution:")
for category in df['gold_category'].value_counts().index:
    count = df['gold_category'].value_counts()[category]
    print(f"{category}: {count} reviews")

print(f"\nThis data demonstrates all policy violation types:")
print("• No_Ads: Promotional codes and contact solicitation")
print("• Irrelevant: Off-topic content unrelated to business")
print("• Rant_No_Visit: Generic negative comments without visit evidence")
print("• None: Legitimate reviews that should be approved")

## 4. Configuration Setup

In [ ]:
# Create configuration classes matching the actual pipeline
config_code = '''
"""
Pipeline Configuration Classes - Matching Production Structure
"""

from dataclasses import dataclass, field
from typing import Dict, List, Optional
import os

@dataclass
class ModelConfig:
    """Configuration for model settings"""
    # HuggingFace models (matching actual pipeline)
    hf_sentiment_model: str = "distilbert-base-uncased-finetuned-sst-2-english"
    hf_toxicity_model: str = "unitary/toxic-bert"
    hf_zero_shot_model: str = "facebook/bart-large-mnli"

    # Gemini configuration
    gemini_model: str = "gemini-2.5-flash-lite"

    # Confidence thresholds (matching actual pipeline)
    sentiment_threshold: float = 0.7
    toxicity_threshold: float = 0.5
    zero_shot_threshold: float = 0.7
    ensemble_tau: float = 0.55

@dataclass
class DataConfig:
    """Configuration for data paths and settings"""
    data_dir: str = "data"
    raw_data_dir: str = "data/raw"
    processed_data_dir: str = "data/clean"  # Matches actual structure
    sample_data_dir: str = "data/sample"
    pseudo_label_dir: str = "data/pseudo-label"  # Matches actual structure
    training_dir: str = "data/training"
    testing_dir: str = "data/testing"

    # Default input file
    sample_reviews_file: str = "data/sample/sample_reviews.csv"

@dataclass
class OutputConfig:
    """Configuration for output paths"""
    results_dir: str = "results"
    predictions_dir: str = "results/predictions"
    evaluations_dir: str = "results/evaluations"
    reports_dir: str = "results/reports"

    # Default output files (matching actual pipeline)
    hf_predictions: str = "results/predictions/predictions_hf.csv"
    ensemble_predictions: str = "results/predictions/predictions_ens.csv"

@dataclass
class PipelineConfig:
    """Main pipeline configuration combining all components"""
    model: ModelConfig = field(default_factory=ModelConfig)
    data: DataConfig = field(default_factory=DataConfig)
    output: OutputConfig = field(default_factory=OutputConfig)

    # Gemini configuration
    gemini_api_key: str = ""

    # Pipeline settings
    batch_size: int = 32
    max_workers: int = 4
    cache_predictions: bool = True
    verbose_logging: bool = True

    def __post_init__(self):
        """Create directories if they don't exist"""
        directories = [
            self.data.raw_data_dir,
            self.data.processed_data_dir,
            self.data.sample_data_dir,
            self.data.pseudo_label_dir,
            self.data.training_dir,
            self.data.testing_dir,
            self.output.predictions_dir,
            self.output.evaluations_dir,
            self.output.reports_dir
        ]

        for directory in directories:
            os.makedirs(directory, exist_ok=True)

# Global configuration instance
config = PipelineConfig()
'''

with open('src/config/pipeline_config.py', 'w') as f:
    f.write(config_code)

print("✅ Configuration created matching production pipeline!")

# Test configuration
exec(config_code)
test_config = PipelineConfig()
print(f"Data directory: {test_config.data.sample_data_dir}")
print(f"HF Zero-shot model: {test_config.model.hf_zero_shot_model}")
print(f"Ensemble tau: {test_config.model.ensemble_tau}")
print(f"Predictions output: {test_config.output.hf_predictions}")

## 5. Constants and Prompts

In [ ]:
# Create constants and prompts matching the actual pipeline
constants_code = '''
"""
Core Constants - Matching Production Pipeline
"""

# Policy Categories (matching actual pipeline)
POLICY_CATEGORIES = {
    'NO_ADS': 'No_Ads',
    'IRRELEVANT': 'Irrelevant',
    'RANT_NO_VISIT': 'Rant_No_Visit',
    'NONE': 'None'
}

# Label Types (matching actual pipeline)
LABELS = {
    'APPROVE': 'APPROVE',
    'REJECT': 'REJECT'
}

# Default Models (matching actual pipeline)
DEFAULT_MODELS = {
    'SENTIMENT': "distilbert-base-uncased-finetuned-sst-2-english",
    'TOXIC': "unitary/toxic-bert",
    'ZERO_SHOT': "facebook/bart-large-mnli",
    'GEMINI_DEFAULT': "gemini-2.5-flash-lite"
}

# Zero-shot Classification Labels (matching actual pipeline)
ZERO_SHOT_LABELS = [
    "an advertisement or promotional solicitation for this business (promo code, referral, links, contact to buy)",
    "off-topic or unrelated to this business (e.g., politics, crypto, chain messages, personal stories not about this place)",
    "a generic negative rant about this business without evidence of a visit (short insults, 'scam', 'overpriced', 'worst ever')",
    "a relevant on-topic description of a visit or experience at this business"
]

# Mapping zero-shot labels to policy categories
ZERO_SHOT_TO_POLICY = {
    ZERO_SHOT_LABELS[0]: POLICY_CATEGORIES['NO_ADS'],
    ZERO_SHOT_LABELS[1]: POLICY_CATEGORIES['IRRELEVANT'],
    ZERO_SHOT_LABELS[2]: POLICY_CATEGORIES['RANT_NO_VISIT'],
    ZERO_SHOT_LABELS[3]: POLICY_CATEGORIES['NONE']
}

# Confidence Thresholds
CONFIDENCE_THRESHOLDS = {
    'HIGH': 0.8,
    'MEDIUM': 0.6,
    'LOW': 0.4,
    'DEFAULT': 0.55
}
'''

with open('src/core/constants.py', 'w') as f:
    f.write(constants_code)

# Create prompt templates (matching actual pipeline)
prompts_code = '''
"""
Policy Prompts - Matching Production Pipeline
"""

# JSON schema all prompts must return
TEMPLATE_JSON = """Return ONLY JSON with no extra text:
{"label":"<APPROVE|REJECT>","category":"<No_Ads|Irrelevant|Rant_No_Visit|None>",
 "rationale":"<short>","confidence":<0.0-1.0>,
 "flags":{"links":false,"coupon":false,"visit_claimed":false}}
"""

# ===== 1) NO ADS / PROMOTIONAL =====
NO_ADS_SYSTEM = """You are a content policy checker for location reviews.
If this specific policy does NOT clearly apply, return APPROVE with category "None" and confidence 0.0. Do not reject for other policies.
Reject ONLY if the review contains clear advertising or promotional solicitation:
- referral/promo/coupon codes, price lists, booking/ordering links, contact-for-order (DM me / WhatsApp / Telegram / email / call), affiliate pitches.
Do NOT mark generic off-topic content (e.g., crypto/politics) as Ads unless it includes explicit solicitation to buy or contact.
Approve normal experiences even if positive or mentioning 'cheap' or 'good deal'.
Output the required JSON only.
"""

# ===== 2) IRRELEVANT CONTENT =====
IRRELEVANT_SYSTEM = """You are checking ONLY for the 'Irrelevant' policy.

Decision rule (mutually exclusive):
- If this specific policy does NOT clearly apply, return APPROVE with category "None" and confidence 0.0.
- Do not reject for other policies (e.g., Ads or Rant_No_Visit).

Reject as Irrelevant when the text is off-topic and unrelated to THIS venue/service:
- unrelated politics/news/crypto hype/chain messages/personal stories
- generic advice not tied to this place (e.g., 'buy BTC now', 'vote X'), etc.
- content about another business or location without discussing this one

Return ONLY JSON with fields: label, category, rationale, confidence (0.0–1.0), flags.
"""

# ===== 3) RANTS WITHOUT VISIT =====
RANT_NO_VISIT_SYSTEM = """Reject generic rants or accusations clearly targeting THIS place but with no evidence of a visit.
These rants are often:
- Short and emotional (e.g., 'Terrible place', 'Worst ever', 'Overpriced scammers')
- Broad accusations ('scam', 'rip-off', 'fraud')
- Negative judgments about pricing, quality, or character of the venue
Reject them even if the reviewer does not explicitly say 'this place/restaurant' — assume negativity is directed at the business being reviewed.
Approve only if the reviewer provides concrete evidence of a visit (date, food ordered, staff interaction).
Output JSON only.
"""

def build_prompt(system_text: str, review_text: str, fewshots):
    demo = "\\n\\n".join(
        [f"Review:\\n{r}\\nExpected JSON:\\n{j}" for r,j in fewshots]
    )
    return f"""{system_text}

{TEMPLATE_JSON}

{demo}

Now classify this review. Return ONLY JSON.

Review:
{review_text}
"""
'''

with open('prompts/policy_prompts.py', 'w') as f:
    f.write(prompts_code)

print("✅ Constants and prompts created matching production pipeline!")

# Test constants
exec(constants_code)
print(f"Policy categories: {list(POLICY_CATEGORIES.values())}")
print(f"Zero-shot model: {DEFAULT_MODELS['ZERO_SHOT']}")
print(f"Default confidence threshold: {CONFIDENCE_THRESHOLDS['DEFAULT']}")
print(f"Zero-shot labels configured: {len(ZERO_SHOT_LABELS)} categories")

## 6. Gemini API Configuration and Pseudo-Labeling

In [ ]:
# Gemini API Key Setup and Pseudo-Labeling Implementation (deduped + cache-aware)
import os
import json
import time
import pandas as pd
from tqdm import tqdm
import google.generativeai as genai

# --- Config & cache paths ---
PSEUDO_DIR = "data/pseudo-label"
os.makedirs(PSEUDO_DIR, exist_ok=True)
PSEUDO_PATH = os.path.join(PSEUDO_DIR, "gemini_pseudo_labels.csv")

def load_cached_pseudo_labels():
    if os.path.exists(PSEUDO_PATH):
        print(f"📦 Loading cached pseudo-labels from {PSEUDO_PATH}")
        return pd.read_csv(PSEUDO_PATH)
    return None

# --- API key (Colab secrets first, manual fallback) ---
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    print("✅ Gemini API key loaded from Colab secrets")
    api_source = "secrets"
except Exception as e:
    print(f"Colab secrets not found: {e}")
    print("Please enter your Gemini API key manually:")
    print("1) https://aistudio.google.com/app/apikey  2) Create key  3) Paste here")
    GEMINI_API_KEY = input("Enter your Gemini API key: ").strip()
    api_source = "manual"

# --- Configure & test Gemini ---
if GEMINI_API_KEY:
    genai.configure(api_key=GEMINI_API_KEY)
    try:
        _test_model = genai.GenerativeModel('gemini-2.5-flash-lite')
        _ = _test_model.generate_content("Test: Say 'API working'")
        print("✅ Gemini API test successful!")
        print(f"   Source: {api_source}")
        print(f"   Model: gemini-2.5-flash-lite")
        gemini_available = True
    except Exception as e:
        print(f"❌ Gemini test failed: {e}")
        print("Check your API key and quota limits")
        gemini_available = False
else:
    print("No API key provided — skipping Gemini pseudo-labeling.")
    gemini_available = False

print("\nConfiguration Summary:")
print(f"   Gemini Available: {'✅ Yes' if gemini_available else '❌ No'}")
print(f"   Pipeline Mode: {'Full' if gemini_available else 'HuggingFace Only'}")

# --- Pseudo-label helpers ---
def classify_with_gemini(text: str, model_name="gemini-2.0-flash-exp"):
    """Call Gemini to classify a review into policy categories."""
    try:
        model = genai.GenerativeModel(model_name)
        prompt = f"""
You are a Google review policy expert. Classify this review and provide a JSON response.

Review: "{text}"

Policy Categories:
- No_Ads: Contains advertisements, promotional content, promo codes, contact information
- Irrelevant: Off-topic content (politics, crypto, unrelated businesses)
- Rant_No_Visit: Generic negative rants without evidence of visiting
- None: Legitimate review about an actual visit/experience

Respond with JSON only:
{{"label": "APPROVE" or "REJECT", "category": "policy_name", "confidence": 0.0-1.0, "rationale": "brief_explanation"}}
"""
        response = model.generate_content(prompt)
        resp = response.text.strip()

        # unwrap code fences if present
        if resp.startswith('```json'):
            resp = resp.replace('```json', '').replace('```', '').strip()
        elif resp.startswith('```'):
            resp = resp.replace('```', '').strip()

        # try strict JSON
        try:
            obj = json.loads(resp)
            if all(k in obj for k in ('label','category','confidence')):
                return {
                    "label": obj['label'],
                    "category": obj['category'],
                    "confidence": float(obj['confidence']),
                    "rationale": obj.get('rationale','Gemini classification')
                }
        except json.JSONDecodeError:
            pass

        # fallback: detect REJECT/APPROVE heuristically
        if 'REJECT' in resp.upper():
            if 'No_Ads' in resp or 'advert' in resp.lower():
                cat = 'No_Ads'
            elif 'Irrelevant' in resp or 'off-topic' in resp.lower():
                cat = 'Irrelevant'
            elif 'Rant_No_Visit' in resp or 'rant' in resp.lower():
                cat = 'Rant_No_Visit'
            else:
                cat = 'No_Ads'
            return {"label":"REJECT","category":cat,"confidence":0.7,"rationale":"Parsed from text"}
        else:
            return {"label":"APPROVE","category":"None","confidence":0.7,"rationale":"Parsed from text"}

    except Exception as e:
        print(f"Gemini API error: {e}")
        return {"label":"APPROVE","category":"None","confidence":0.0,"rationale":f"API error: {e}"}

def generate_pseudo_labels_with_gemini(unlabeled_df, confidence_threshold=0.8, max_labels=50):
    """Generate high-confidence pseudo-labels and save to cache."""
    if not gemini_available:
        print("❌ Gemini not available for pseudo-labeling")
        return pd.DataFrame()

    print("Generating pseudo-labels for training data…")
    print(f"  Confidence threshold: {confidence_threshold}")
    print(f"  Max labels: {max_labels}")

    rows = []
    processed = 0
    total = min(len(unlabeled_df), max_labels)
    print(f"Processing {total} reviews…")

    for _, row in tqdm(unlabeled_df.head(total).iterrows(), total=total, desc="Generating pseudo-labels"):
        txt = str(row['text'])
        res = classify_with_gemini(txt)
        if res['confidence'] >= confidence_threshold:
            rows.append({
                'id': row.get('id', processed + 100),
                'text': txt,
                'pred_label': res['label'],
                'pred_category': res['category'],
                'confidence': res['confidence'],
                'rationale': res['rationale'],
                'source': 'gemini_pseudo'
            })
        processed += 1
        time.sleep(0.1)  # rate limit

    out = pd.DataFrame(rows)
    if len(out) > 0:
        out.to_csv(PSEUDO_PATH, index=False)
        print(f"✅ Generated {len(out)} pseudo-labels")
        print(f"💾 Cached to {PSEUDO_PATH}")
        try:
            print(f"Label distribution: {out['pred_label'].value_counts().to_dict()}")
            print(f"Category distribution: {out['pred_category'].value_counts().to_dict()}")
            print(f"Average confidence: {out['confidence'].mean():.3f}")
        except Exception:
            pass
    else:
        print("❌ No high-confidence pseudo-labels generated")
    return out

# --- Use cache if present; otherwise, generate ---
cached = load_cached_pseudo_labels()
if cached is not None and len(cached) > 0:
    pseudo_labels_df = cached
    print(f"✅ Using cached pseudo-labels (n={len(pseudo_labels_df)})")
else:
    if gemini_available:
        print("\n" + "="*60)
        print("GENERATING TRAINING DATA WITH PSEUDO-LABELS")
        print("="*60)
        unlabeled_df = pd.DataFrame({
            'id': list(range(101, 121)),
            'text': [
                "Amazing food and service, definitely coming back!",
                "Visit our website for exclusive deals and discounts - use code SAVE20",
                "The worst experience ever, everything was terrible, total scam",
                "Staff was friendly, food was fresh and tasty, good value for money",
                "This place is overpriced, never going back, waste of money",
                "Great atmosphere, perfect for family dinner, ordered the set meal and dessert",
                "Follow my Instagram @foodie123 for more reviews and promos",
                "Bitcoin is going to the moon! Buy now before it's too late!",
                "Had the chicken rice here yesterday, portion was generous and taste was authentic",
                "DM me for discount codes! Also selling crypto courses online",
                "Terrible service, rude staff, food was cold when it arrived",
                "The laksa here reminds me of my grandmother's cooking, very nostalgic",
                "Check out my YouTube channel for food reviews and crypto tips",
                "Went here for lunch with colleagues, everyone enjoyed their meals",
                "Overpriced tourist trap, locals know better places nearby",
                "Made reservation for 6pm, got seated immediately, excellent service throughout",
                "Politics in this country is corrupt, restaurants like this are part of the problem",
                "Their signature dish was perfectly seasoned, will definitely recommend to friends",
                "Worst restaurant in Singapore, total ripoff, avoid at all costs",
                "Celebrated my birthday here last week, staff even brought out a cake"
            ]
        })
        pseudo_labels_df = generate_pseudo_labels_with_gemini(
            unlabeled_df, confidence_threshold=0.7, max_labels=20
        )
    else:
        print("\n" + "="*60)
        print("SKIPPING PSEUDO-LABELING (Gemini not available)")
        print("="*60)
        pseudo_labels_df = pd.DataFrame()

print(f"\nTraining data preparation: {'✅ Complete' if len(pseudo_labels_df) > 0 else '❌ Skipped'}")
print(f"Ready for HuggingFace model training: {'✅ Yes' if len(pseudo_labels_df) > 0 else '⚠️ Pre-trained only'}")



## 7. HuggingFace Model Training with Pseudo-Labels

In [ ]:
!pip uninstall -y transformers -q
!pip install -qU "transformers>=4.45.0" "accelerate>=0.34.0" "huggingface_hub>=0.23.0"

# 7. HuggingFace Model Training with Pseudo-Labels
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, DataCollatorWithPadding, pipeline as hf_pipeline,
    __version__ as HF_VER,   # <-- for version-aware args
)
from packaging.version import parse                # <-- for version-aware args
from datasets import Dataset
import torch, numpy as np, pandas as pd, re, os
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm

print("HUGGINGFACE MODEL TRAINING WITH PSEUDO-LABELS (no sentiment, fused policy logic)")
print("=" * 60)

# ---- import your constants (as provided)
from src.core.constants import (
    DEFAULT_MODELS, ZERO_SHOT_LABELS, ZERO_SHOT_TO_POLICY,
    POLICY_CATEGORIES, LABELS, CONFIDENCE_THRESHOLDS
)

import os, pandas as pd


PSEUDO_DIR = "data/pseudo-label"
PSEUDO_PATH = os.path.join(PSEUDO_DIR, "gemini_pseudo_labels.csv")
os.makedirs(PSEUDO_DIR, exist_ok=True)

def load_cached_pseudo_labels(path=PSEUDO_PATH):
    if os.path.exists(path):
        print(f"📦 Loading cached pseudo-labels from {path}")
        df_cached = pd.read_csv(path)
        need = {"id","text","pred_label","pred_category","confidence"}
        missing = need - set(df_cached.columns)
        if missing:
            raise ValueError(f"Cached pseudo-label file missing columns: {missing}")
        print(f"✅ Loaded {len(df_cached)} pseudo-labeled rows")
        return df_cached
    print(f"ℹ️ No cache at {path}.")
    return pd.DataFrame(columns=["id","text","pred_label","pred_category","confidence"])

# make pseudo_labels_df available for training-mode decision
pseudo_labels_df = load_cached_pseudo_labels()

# decide mode only AFTER loading the cache
device = 0 if torch.cuda.is_available() else -1
has_training_data = len(pseudo_labels_df) > 0
training_mode = "fine-tuning" if has_training_data else "pre-trained"
print(f"Mode: {training_mode}  |  Pseudo-label rows: {len(pseudo_labels_df)}")

# ---- models (NO sentiment)
BASE_MODEL      = "distilbert-base-uncased"
TOXIC_MODEL     = DEFAULT_MODELS['TOXIC']
ZERO_SHOT_MODEL = DEFAULT_MODELS['ZERO_SHOT']

# ========= new policy helpers (ad evidence + toxicity gate) =========
AD_PATTERNS = [
    r"https?://", r"\bwww\.", r"\.[a-z]{2,6}\b",
    r"\b(?:\+?\d[\s\-()]*){7,}\b",
    r"\bpromo(?:\s*code)?\b", r"\bdiscount\b", r"\bcoupon\b",
    r"\breferral\b", r"\buse\s*code\b", r"\benter\s*code\b",
    r"\bwhatsapp\b",
    r"\bdm\s+(?:me|us)\b",
    r"\bcontact\s+(?:us|me)\b", r"\bcall\s+(?:us|me)\b",
]
AD_REGEX = re.compile("|".join(AD_PATTERNS), flags=re.IGNORECASE)
TOX_TO_RANT = {"toxic", "severe_toxic", "obscene", "threat", "insult"}
TOX_TO_IRRELEVANT = {"identity_hate"}

def ad_evidence(text: str):
    t = text or ""
    m = AD_REGEX.search(t)
    return bool(m), (m.group(0) if m else "")

def tox_top_label(tox_output):
    """
    Normalize HF toxicity pipeline outputs into (label, score).
    Handles dict, list[dict], list[list[dict]].
    """
    try:
        if isinstance(tox_output, dict):
            candidates = [tox_output]
        elif isinstance(tox_output, list):
            if len(tox_output) and isinstance(tox_output[0], dict):
                candidates = tox_output
            elif len(tox_output) and isinstance(tox_output[0], list):
                candidates = tox_output[0]
            else:
                candidates = []
        else:
            candidates = []
        if not candidates:
            return "NONE", 0.0
        best = max(candidates, key=lambda d: float(d.get("score", 0.0)))
        return best.get("label", "NONE"), float(best.get("score", 0.0))
    except Exception:
        return "NONE", 0.0


def policy_zero_shot_fused(zshot, toxic, text: str,
                           tau_irrelevant=0.55, tau_rant=0.55,
                           tau_ads=0.70, tox_tau=0.50, ads_margin=0.10):
    """
    1) Toxicity gate:
         - identity_hate -> Irrelevant
         - toxic/insult/obscene/threat -> Rant_No_Visit
    2) Else if Irrelevant/Rant over thresholds -> that category
    3) Else if Ads high + ad evidence + margin -> No_Ads
    4) Else -> None
    Returns: (pred_label 'APPROVE'/'REJECT', pred_category)
    """
    # zero-shot on your label set
    zs_res = zshot(text, candidate_labels=ZERO_SHOT_LABELS,
                   hypothesis_template="This review is {}.", multi_label=True)
    zs = {lab: float(scr) for lab, scr in zip(zs_res["labels"], zs_res["scores"])}
    ads  = zs.get(ZERO_SHOT_LABELS[0], 0.0)
    irr  = zs.get(ZERO_SHOT_LABELS[1], 0.0)
    rant = zs.get(ZERO_SHOT_LABELS[2], 0.0)
    none = zs.get(ZERO_SHOT_LABELS[3], 0.0)

    # toxicity top label
    tox_label, tox_score = tox_top_label(toxic(text))


    if tox_label and tox_score >= tox_tau:
        if tox_label in TOX_TO_RANT:
            return LABELS['REJECT'], POLICY_CATEGORIES['RANT_NO_VISIT']
        if tox_label in TOX_TO_IRRELEVANT:
            return LABELS['REJECT'], POLICY_CATEGORIES['IRRELEVANT']

    if max(irr, rant) >= min(tau_irrelevant, tau_rant):
        return LABELS['REJECT'], (POLICY_CATEGORIES['IRRELEVANT'] if irr >= rant else POLICY_CATEGORIES['RANT_NO_VISIT'])

    has_ads, _ = ad_evidence(text)
    if has_ads and (ads >= tau_ads) and (ads >= max(irr, rant) + ads_margin):
        return LABELS['REJECT'], POLICY_CATEGORIES['NO_ADS']

    return LABELS['APPROVE'], POLICY_CATEGORIES['NONE']

# ========= metrics & training (binary, same as before) =========
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted', zero_division=0)
    acc = accuracy_score(labels, predictions)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

def prepare_training_data(pseudo_df):
    train_texts  = pseudo_df['text'].tolist()
    train_labels = [1 if label == 'REJECT' else 0 for label in pseudo_df['pred_label']]
    return train_texts, train_labels

def train_custom_classification_model(train_texts, train_labels, model_name="review-policy-classifier"):
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
    model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=2)
    enc = tokenizer(train_texts, truncation=True, padding=True, max_length=256, return_tensors="pt")
    train_ds = Dataset.from_dict({'input_ids': enc['input_ids'], 'attention_mask': enc['attention_mask'], 'labels': train_labels})

    # ---- version-aware TrainingArguments (v4 vs v5)
    base_kwargs = dict(
        output_dir=f'./models/fine-tuned/{model_name}',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=100,
        weight_decay=0.01,
        logging_dir=f'./logs/{model_name}',
        logging_steps=10,
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
        report_to="none",
    )

    try:
        # Transformers v5+
        args = TrainingArguments(**base_kwargs, eval_strategy="epoch")
    except TypeError:
        # Transformers v4.x
        args = TrainingArguments(**base_kwargs, evaluation_strategy="epoch")


    trainer = Trainer(
        model=model, args=args,
        train_dataset=train_ds, eval_dataset=train_ds,
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics,
    )
    print("Starting training...")
    trainer.train()
    save_dir = f'./models/fine-tuned/{model_name}'
    trainer.save_model(save_dir)
    tokenizer.save_pretrained(save_dir)
    print(f"✅ Model saved to: {save_dir}")
    return model, tokenizer, save_dir

def load_aux_pipelines(device=None):
    toxic = hf_pipeline("text-classification", model=TOXIC_MODEL, top_k=None, device=device)
    zshot = hf_pipeline("zero-shot-classification", model=ZERO_SHOT_MODEL, device=device)
    return toxic, zshot

def predict_with_custom_model(text, model, tokenizer):
    inputs = tokenizer(text, truncation=True, padding=True, max_length=256, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.softmax(logits, dim=-1)
        conf = float(probs.max())
        pred = int(probs.argmax())
    return ("REJECT" if pred == 1 else "APPROVE"), conf

def run_inference_pipeline(df, tau=CONFIDENCE_THRESHOLDS['DEFAULT']):
    print(f"\nRunning inference… ({training_mode})  samples={len(df)}")
    toxic_pipeline, zshot_pipeline = load_aux_pipelines(device)
    results = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing reviews"):
        txt = str(row['text'])

        # ---- robust access to optional custom model
        cc = TRAINED_MODELS.get('custom_classifier')
        if cc is not None:
            pred_label, confidence = predict_with_custom_model(txt, cc['model'], cc['tokenizer'])
            pred_category = "Custom_Trained"
        else:
            pred_label, pred_category = policy_zero_shot_fused(
                zshot=zshot_pipeline, toxic=toxic_pipeline, text=txt,
                tau_irrelevant=0.55, tau_rant=0.55, tau_ads=0.70, tox_tau=0.50, ads_margin=0.10
            )
            confidence = None  # rule-based fusion

        # Optional: record toxicity top label/score for debugging
        try:
            tox_label, tox_score = tox_top_label(toxic_pipeline(txt))
        except Exception:
            tox_label, tox_score = "NONE", 0.0

        results.append({
            "id": row.get('id', len(results)+1),
            "text": txt,
            "pred_label": pred_label,
            "pred_category": pred_category,
            "confidence": (round(float(confidence), 4) if confidence is not None else None),
            "toxicity_label": tox_label,
            "toxicity_score": round(float(tox_score), 4),
            "model_type": training_mode
        })

    results_df = pd.DataFrame(results)
    os.makedirs("results/predictions", exist_ok=True)
    out_path = f'results/predictions/predictions_{training_mode}.csv'
    results_df.to_csv(out_path, index=False)
    print(f"✅ Inference saved → {out_path}  rows={len(results_df)}")
    return results_df

# ========= Train (if pseudo-labels) or use pre-trained =========
TRAINED_MODELS = {"custom_classifier": None}   # <-- init key to avoid KeyError
if training_mode == "fine-tuning":
    print("\n🎯 FINE-TUNING MODE")
    tr_texts, tr_labels = prepare_training_data(pseudo_labels_df)
    model, tok, model_path = train_custom_classification_model(tr_texts, tr_labels, "review-policy-classifier")
    TRAINED_MODELS['custom_classifier'] = {'model_path': model_path, 'model': model, 'tokenizer': tok}
else:
    print("\n🔄 PRE-TRAINED MODE (no sentiment)")
    TRAINED_MODELS['custom_classifier'] = None

# ========= Test on available data (same style) =========
print("\n" + "="*60)
print("TESTING MODELS")
print("="*60)

if len(pseudo_labels_df) > 0:
    test_data = pseudo_labels_df[['id','text']]
    src = f"Pseudo-labeled data (n={len(test_data)})"
else:
    # fallback minimal sample
    test_data = pd.DataFrame({'id':[1,2], 'text':[
        "Great food and service!",
        "Use my promo code SAVE20 for discounts!"
    ]})
    src = "Minimal test data"

print(f"Testing with: {src}")
hf_results = run_inference_pipeline(test_data, tau=CONFIDENCE_THRESHOLDS['DEFAULT'])

# Pretty print head
print("\n📊 INFERENCE RESULTS")
print("="*50)
disp_cols = ['id','text','pred_label','pred_category','confidence','model_type','toxicity_label','toxicity_score']
dd = hf_results.copy()
dd['text'] = dd['text'].apply(lambda x: x[:50]+"..." if isinstance(x,str) and len(x)>50 else x)
print(dd[disp_cols].head(10).to_string(index=False))

# Summary
print("\n📈 RESULTS SUMMARY")
print(hf_results['pred_label'].value_counts())
print(f"Average Confidence (fine-tuned only): {hf_results['confidence'].dropna().mean() if 'confidence' in hf_results else float('nan'):.3f}")

print("\n✅ MODEL TRAINING AND TESTING COMPLETE")
print(f"Training Mode: {training_mode.upper()}")
print("Ready for Model Persistence: ✅ Yes")


## 8. Unified Spam Detection Model


In [ ]:
"""
Unified spam detection system combining machine learning with pattern analysis.
Assumes data has already passed through toxicity/hate speech filtering pipeline.
"""

import pandas as pd
import numpy as np
import re
from typing import Dict, List, Tuple, Optional, Union
from dataclasses import dataclass
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report, confusion_matrix

!pip install textstat
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

@dataclass
class DetectionResult:
    """Container for detection results."""
    label: str  # 'APPROVE' or 'REJECT'
    confidence: float  # 0.0 to 1.0
    category: str  # violation category or 'None'
    features: Dict  # method-specific features
    confidence_interval: Tuple[float, float]

class PatternFeatureExtractor(BaseEstimator, TransformerMixin):
    """Extract pattern-based features for ML model."""

    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        """Extract pattern features from texts."""
        features = []

        for text in X:
            text_features = self._extract_pattern_features(text)

            # Convert to feature vector
            feature_vector = [
                text_features['repetition_ratio'],
                text_features['word_diversity_ratio'],
                text_features['repeated_ngrams_ratio'],
                text_features['caps_ratio'],
                text_features['punct_ratio'],
                text_features['readability_score'] / 100.0,  # Normalize
                text_features['avg_sentence_length'] / 50.0,  # Normalize
                text_features['word_count'] / 100.0,  # Normalize
                text_features['template_score'],
                text_features['phrase_repetition_score'],
                text_features['local_repetition_score']
            ]
            features.append(feature_vector)

        return np.array(features)

    def _extract_pattern_features(self, text: str) -> Dict:
        """Extract comprehensive pattern features from text."""
        words = text.split()
        word_count = len(words)
        char_count = len(text)

        # Basic repetition analysis
        word_freq = {}
        for word in words:
            word_lower = word.lower()
            word_freq[word_lower] = word_freq.get(word_lower, 0) + 1

        repetition_ratio = 0.0
        if word_count > 0:
            max_repetition = max(word_freq.values()) if word_freq else 1
            repetition_ratio = max_repetition / word_count

        # Word diversity (unique words / total words)
        unique_words = len(set(word.lower() for word in words))
        word_diversity_ratio = unique_words / max(word_count, 1)

        # N-gram repetition detection (for "food is good food is great food is nice")
        repeated_ngrams_ratio = self._detect_repeated_ngrams(words)

        # Phrase repetition score (detects patterns like "food is X" repeating)
        phrase_repetition_score = self._detect_phrase_patterns(words)

        # Local repetition score (detects repetition in specific text segments)
        local_repetition_score = self._detect_local_repetition(words)

        # Template detection (common patterns)
        template_indicators = [
            r'\b(excellent|amazing|great|good|bad|terrible)\b.*\b(food|service|place|restaurant)\b',
            r'\b(recommend|suggest|try|visit)\b.*\b(place|restaurant|here)\b',
            r'\b(will|would).*\b(come|go|visit).*\b(again|back)\b'
        ]

        template_matches = sum(1 for pattern in template_indicators
                             if re.search(pattern, text, re.IGNORECASE))
        template_score = template_matches / len(template_indicators)

        # Simple readability proxy (no textstat installed)
        words = len(text.split())
        sentences = len(re.split(r'[.!?]+', text))
        if sentences > 0 and words > 0:
            avg_words_per_sentence = words / sentences
            readability_score = max(0, min(100, 120 - avg_words_per_sentence * 2))
        else:
            readability_score = 50.0


        # Character-level features
        caps_count = sum(1 for c in text if c.isupper())
        punct_count = sum(1 for c in text if c in '!?.,;:')

        caps_ratio = caps_count / max(char_count, 1)
        punct_ratio = punct_count / max(char_count, 1)

        # Sentence structure
        sentences = re.split(r'[.!?]+', text)
        sentence_count = len([s for s in sentences if s.strip()])
        avg_sentence_length = word_count / max(sentence_count, 1)

        return {
            'word_count': word_count,
            'char_count': char_count,
            'repetition_ratio': repetition_ratio,
            'word_diversity_ratio': word_diversity_ratio,
            'repeated_ngrams_ratio': repeated_ngrams_ratio,
            'phrase_repetition_score': phrase_repetition_score,
            'local_repetition_score': local_repetition_score,
            'template_score': template_score,
            'readability_score': readability_score,
            'caps_ratio': caps_ratio,
            'punct_ratio': punct_ratio,
            'sentence_count': sentence_count,
            'avg_sentence_length': avg_sentence_length
        }

    def _detect_repeated_ngrams(self, words: List[str], n: int = 2) -> float:
        """Detect repeated n-grams that indicate spam patterns."""
        if len(words) < n * 2:  # Need at least 2 n-grams to compare
            return 0.0

        # Generate n-grams
        ngrams = []
        for i in range(len(words) - n + 1):
            ngram = ' '.join(words[i:i+n]).lower()
            ngrams.append(ngram)

        if not ngrams:
            return 0.0

        # Count n-gram frequencies
        ngram_freq = {}
        for ngram in ngrams:
            ngram_freq[ngram] = ngram_freq.get(ngram, 0) + 1

        # Calculate repetition score
        total_ngrams = len(ngrams)
        repeated_ngrams = sum(1 for freq in ngram_freq.values() if freq > 1)

        return repeated_ngrams / total_ngrams if total_ngrams > 0 else 0.0

    def _detect_phrase_patterns(self, words: List[str]) -> float:
        """Detect repeating phrase patterns like 'food is X food is Y food is Z'."""
        if len(words) < 6:  # Need enough words for pattern detection
            return 0.0

        # Look for patterns where same 2-word prefix repeats
        pattern_scores = []

        # Check 2-word patterns
        for i in range(len(words) - 3):
            prefix = f"{words[i]} {words[i+1]}".lower()

            # Count how many times this prefix appears
            prefix_count = 0
            for j in range(i, len(words) - 1):
                if j + 1 < len(words):
                    candidate = f"{words[j]} {words[j+1]}".lower()
                    if candidate == prefix:
                        prefix_count += 1

            if prefix_count > 1:
                # This prefix repeats - calculate pattern strength
                pattern_strength = prefix_count / (len(words) / 3)  # Normalize by text length
                pattern_scores.append(min(pattern_strength, 1.0))

        # Also check for local repetition at the end of text (common spam pattern)
        local_repetition_score = self._detect_local_repetition(words)

        return max(max(pattern_scores) if pattern_scores else 0.0, local_repetition_score)

    def _detect_local_repetition(self, words: List[str]) -> float:
        """Detect repetitive patterns in specific sections of text (e.g., end of text)."""
        if len(words) < 6:
            return 0.0

        max_score = 0.0

        # Check multiple segments: end of text and any suspicious consecutive patterns
        segments_to_check = []

        # 1. End segment (common spam pattern)
        end_segment_size = min(10, max(6, len(words) // 3))
        end_words = words[-end_segment_size:]
        segments_to_check.append(('end', end_words))

        # 2. Look for any consecutive repeated phrases throughout the text
        for start_pos in range(len(words) - 6):  # Sliding window
            window_size = min(8, len(words) - start_pos)
            window_words = words[start_pos:start_pos + window_size]
            segments_to_check.append(('window', window_words))

        for segment_type, segment_words in segments_to_check:
            # Look for exact phrase repetition in this segment
            for phrase_len in [2, 3, 4]:  # Check 2-word, 3-word, 4-word phrases
                if len(segment_words) < phrase_len * 2:  # Need at least 2 instances
                    continue

                for i in range(len(segment_words) - phrase_len + 1):
                    phrase = ' '.join(segment_words[i:i+phrase_len]).lower()

                    # Look for consecutive repetition (more suspicious than scattered)
                    consecutive_count = 1
                    next_pos = i + phrase_len

                    while next_pos + phrase_len <= len(segment_words):
                        next_phrase = ' '.join(segment_words[next_pos:next_pos+phrase_len]).lower()
                        if next_phrase == phrase:
                            consecutive_count += 1
                            next_pos += phrase_len
                        else:
                            break

                    if consecutive_count >= 2:  # Found consecutive repeated phrase
                        # Higher score for consecutive repetition
                        if segment_type == 'end':
                            # End segment repetition is more suspicious
                            local_score = (consecutive_count * phrase_len * 1.5) / len(segment_words)
                        else:
                            # General repetition
                            local_score = (consecutive_count * phrase_len) / len(segment_words)

                        max_score = max(max_score, min(local_score, 1.0))

        return max_score


class UnifiedSpamDetector:
    """Unified spam detector combining TF-IDF and pattern analysis in single ML model."""

    def __init__(self, max_features: int = 5000, ngram_range: Tuple[int, int] = (1, 3),
                 spam_threshold: float = 0.3):
        self.max_features = max_features
        self.ngram_range = ngram_range
        self.spam_threshold = spam_threshold
        self.pipeline = None
        self.calibrated_pipeline = None
        self.training_size = 0

    def fit(self, texts: List[str], labels: List[str], calibrate: bool = True):
        """
        Fit the unified model combining TF-IDF and pattern features.

        Args:
            texts: List of review texts
            labels: List of labels ('APPROVE' or 'REJECT')
            calibrate: Whether to calibrate probabilities
        """
        # Convert labels to binary
        y = [1 if label == 'REJECT' else 0 for label in labels]
        self.training_size = len(texts)

        # For small datasets, disable ML-based classification and rely on patterns only
        if len(texts) < 10:
            # With very small training data, ML is unreliable - use pattern-only mode
            self.effective_threshold = 0.9  # Very high threshold to disable ML classification
            self.pattern_only_mode = True
            print(f"⚠️  Very small training set ({len(texts)} samples). Using pattern-only detection mode.")
        elif len(texts) < 20:
            # Small dataset - be more conservative with ML threshold
            adjusted_threshold = min(0.6, self.spam_threshold + 0.2)
            print(f"⚠️  Small training set ({len(texts)} samples). Adjusting threshold: {self.spam_threshold:.2f} → {adjusted_threshold:.2f}")
            self.effective_threshold = adjusted_threshold
            self.pattern_only_mode = False
        else:
            self.effective_threshold = self.spam_threshold
            self.pattern_only_mode = False

        # Create unified pipeline with both TF-IDF and pattern features
        tfidf_vectorizer = TfidfVectorizer(
            max_features=self.max_features,
            ngram_range=self.ngram_range,
            stop_words='english',
            lowercase=True,
            min_df=1,
            max_df=0.95
        )

        pattern_extractor = PatternFeatureExtractor()

        # Combine features using FeatureUnion
        feature_union = FeatureUnion([  # type: ignore
            ('tfidf', tfidf_vectorizer),
            ('patterns', pattern_extractor)
        ])

        self.pipeline = Pipeline([
            ('features', feature_union),
            ('classifier', LogisticRegression(
                random_state=42,
                max_iter=1000,
                class_weight='balanced'
            ))
        ])

        # Fit the pipeline
        print("Training unified ML + Pattern model...")
        self.pipeline.fit(texts, y)

        # Calibrate probabilities if requested
        if calibrate and len(texts) >= 10:
            min_class_size = min(sum(y), len(y) - sum(y))
            cv_folds = min(3, max(2, min_class_size))

            self.calibrated_pipeline = CalibratedClassifierCV(
                self.pipeline,
                method='isotonic',
                cv=cv_folds
            )
            self.calibrated_pipeline.fit(texts, y)  # type: ignore
        elif calibrate:
            print(f"Skipping calibration: need at least 10 samples, got {len(texts)}")

    def predict(self, texts: List[str], use_calibrated: bool = True) -> List[DetectionResult]:
        """Predict spam using unified model."""
        if self.pipeline is None:
            raise ValueError("Model not fitted. Call fit() first.")

        pipeline = self.calibrated_pipeline if (use_calibrated and self.calibrated_pipeline) else self.pipeline

        # Get predictions and probabilities
        proba = pipeline.predict_proba(texts)  # type: ignore
        spam_proba = proba[:, 1]  # Probability of spam (REJECT)

        results = []
        for i, prob in enumerate(spam_proba):
            # Extract pattern features for analysis
            pattern_extractor = PatternFeatureExtractor()
            text_features = pattern_extractor._extract_pattern_features(texts[i])

            # Check if we're in pattern-only mode (small training data)
            pattern_only_mode = getattr(self, 'pattern_only_mode', False)
            threshold = getattr(self, 'effective_threshold', self.spam_threshold)

            if pattern_only_mode:
                # Pattern-only classification for small datasets
                word_count = text_features['word_count']

                # Define clear spam patterns
                is_obvious_spam = (
                    (text_features['repetition_ratio'] > 0.8) or  # Very high word repetition
                    (text_features['phrase_repetition_score'] > 0.7) or  # Very high phrase repetition
                    (text_features['repeated_ngrams_ratio'] > 0.6) or  # Very high ngram repetition
                    (text_features['local_repetition_score'] > 0.4) or  # Local repetition (like "food is good food is good")
                    (word_count < 3 and text_features['repetition_ratio'] > 0.6)  # Short repetitive text
                )

                # Special cases for high-confidence spam detection
                is_phrase_spam = text_features['phrase_repetition_score'] > 0.9
                is_local_spam = text_features['local_repetition_score'] > 0.5

                # Additional filters to avoid false positives (but not for phrase spam)
                seems_legitimate = (
                    text_features['readability_score'] > 30 and
                    word_count > 8 and
                    text_features['word_diversity_ratio'] > 0.6 and
                    text_features['phrase_repetition_score'] < 0.8  # Allow phrase detection override
                )

                # Flag obvious spam patterns, high phrase repetition, or local repetition
                if is_phrase_spam or is_local_spam or (is_obvious_spam and not seems_legitimate):
                    label = 'REJECT'
                    confidence = max(
                        text_features['repetition_ratio'],
                        text_features['phrase_repetition_score'],
                        text_features['repeated_ngrams_ratio'],
                        text_features['local_repetition_score']
                    )
                else:
                    label = 'APPROVE'
                    confidence = 1.0 - max(
                        text_features['repetition_ratio'] * 0.5,
                        text_features['phrase_repetition_score'] * 0.5,
                        text_features['repeated_ngrams_ratio'] * 0.5,
                        text_features['local_repetition_score'] * 0.5
                    )

                pattern_override = (label == 'REJECT')

            else:
                # Normal ML + pattern mode
                label = 'REJECT' if prob > threshold else 'APPROVE'

                # Pattern-based override for obvious spam
                word_count = text_features['word_count']
                is_very_repetitive = (
                    (text_features['repetition_ratio'] > 0.8) or
                    (text_features['phrase_repetition_score'] > 0.7) or
                    (text_features['repeated_ngrams_ratio'] > 0.6)
                )

                has_reasonable_diversity = text_features['word_diversity_ratio'] > 0.3 and word_count > 6
                is_likely_review = text_features['readability_score'] > 20 and word_count > 4

                if is_very_repetitive and not (has_reasonable_diversity and is_likely_review):
                    label = 'REJECT'
                    pattern_override = True
                    confidence = max(
                        text_features['repetition_ratio'],
                        text_features['phrase_repetition_score'],
                        text_features['repeated_ngrams_ratio']
                    )
                else:
                    pattern_override = False
                    confidence = prob if label == 'REJECT' else (1 - prob)

            # Determine category based on dominant pattern
            category = 'None'
            if label == 'REJECT':
                if pattern_override:
                    if text_features['repetition_ratio'] > 0.6:
                        category = 'Repetitive_Spam'
                    elif text_features['local_repetition_score'] > 0.4:
                        category = 'Local_Repetition_Spam'
                    elif text_features['phrase_repetition_score'] > 0.5:
                        category = 'Phrase_Pattern_Spam'
                    elif text_features['repeated_ngrams_ratio'] > 0.4:
                        category = 'NGram_Pattern_Spam'
                    else:
                        category = 'Pattern_Spam'
                else:
                    # ML-detected spam
                    if text_features['repetition_ratio'] > 0.4 or text_features['phrase_repetition_score'] > 0.4:
                        category = 'Repetitive_Spam'
                    elif text_features['template_score'] > 0.6:
                        category = 'Template_Spam'
                    else:
                        category = 'ML_Detected_Spam'

            # Simple confidence interval calculation
            ci_margin = 0.1 * (1 - confidence)  # Smaller margin for higher confidence
            ci_lower = max(0.0, confidence - ci_margin)
            ci_upper = min(1.0, confidence + ci_margin)

            results.append(DetectionResult(
                label=label,
                confidence=float(confidence),
                category=category,
                features={
                    'spam_probability': float(prob),
                    'pattern_features': text_features,
                    'threshold_used': float(threshold)
                },
                confidence_interval=(float(ci_lower), float(ci_upper))
            ))

        return results

    def get_feature_importance(self, top_k: int = 20) -> List[Tuple[str, float]]:
        """Get most important features from the unified model."""
        if self.pipeline is None:
            raise ValueError("Model not fitted.")

        try:
            classifier = self.pipeline.named_steps['classifier']
            feature_union = self.pipeline.named_steps['features']

            # Get feature names
            tfidf_vectorizer = feature_union.transformer_list[0][1]
            tfidf_names = list(tfidf_vectorizer.get_feature_names_out())
            pattern_names = [
                'repetition_ratio', 'word_diversity_ratio', 'repeated_ngrams_ratio',
                'caps_ratio', 'punct_ratio', 'readability_score',
                'avg_sentence_length', 'word_count', 'template_score',
                'phrase_repetition_score', 'local_repetition_score'
            ]

            all_feature_names = tfidf_names + pattern_names

            # Get coefficients
            coef = classifier.coef_[0]
            top_indices = np.argsort(np.abs(coef))[-top_k:][::-1]

            return [(all_feature_names[idx], float(coef[idx])) for idx in top_indices]
        except Exception as e:
            print(f"Warning: Could not extract feature importance: {e}")
            return []


def load_training_data(file_path: str) -> Tuple[List[str], List[str], List[str]]:
    """Load training data from CSV file."""
    df = pd.read_csv(file_path)

    # Handle different column name variations
    text_col = None
    for col in ['text', 'review', 'content', 'text_clean']:
        if col in df.columns:
            text_col = col
            break

    if text_col is None:
        raise ValueError(f"No text column found. Available columns: {list(df.columns)}")

    texts = df[text_col].astype(str).tolist()

    # Handle labels - check if column exists, if not create default
    if 'gold_label' in df.columns:
        labels = df['gold_label'].tolist()
    elif 'label' in df.columns:
        labels = df['label'].tolist()
    else:
        labels = ['APPROVE'] * len(texts)

    # Handle categories - check if column exists, if not create default
    if 'gold_category' in df.columns:
        categories = df['gold_category'].tolist()
    elif 'category' in df.columns:
        categories = df['category'].tolist()
    else:
        categories = ['None'] * len(texts)

    return texts, labels, categories


def evaluate_detector(detector: UnifiedSpamDetector,
                              texts: List[str],
                              true_labels: List[str],
                              true_categories: List[str]) -> Dict:
    """Evaluate the unified detector performance."""
    results = detector.predict(texts)

    # Extract predictions
    pred_labels = [r.label for r in results]
    pred_categories = [r.category for r in results]

    # Binary classification metrics
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score

    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels, pred_labels, average='binary', pos_label='REJECT'
    )
    accuracy = accuracy_score(true_labels, pred_labels)

    # Confidence analysis
    confidences = [r.confidence for r in results]

    evaluation_results = {
        'binary_classification': {
            'accuracy': float(accuracy),
            'precision': float(precision),
            'recall': float(recall),
            'f1': float(f1)
        },
        'confidence_stats': {
            'mean': float(np.mean(confidences)),
            'std': float(np.std(confidences)),
            'min': float(np.min(confidences)),
            'max': float(np.max(confidences))
        }
    }

    return evaluation_results

In [ ]:
# Ensure a dataframe `df` with columns: text, gold_label, gold_category
import pandas as pd, os

PSEUDO_DIR  = "data/pseudo-label"
PSEUDO_PATH = os.path.join(PSEUDO_DIR, "gemini_pseudo_labels.csv")

def choose_training_df():
    # 1) Prefer cached pseudo-labels if present
    if os.path.exists(PSEUDO_PATH):
        print(f"📦 Using cached pseudo-labels → {PSEUDO_PATH}")
        tdf = pd.read_csv(PSEUDO_PATH)
        # normalize column names from pseudo-labels
        rename_map = {
            "pred_label": "gold_label",
            "pred_category": "gold_category"
        }
        for k, v in rename_map.items():
            if k in tdf.columns and v not in tdf.columns:
                tdf = tdf.rename(columns={k: v})
        # sanity check
        need = {"text", "gold_label", "gold_category"}
        missing = need - set(tdf.columns)
        if missing:
            raise ValueError(f"Pseudo-label file missing columns: {missing}")
        # ensure types
        tdf["text"] = tdf["text"].astype(str)
        tdf["gold_label"] = tdf["gold_label"].astype(str)
        tdf["gold_category"] = tdf["gold_category"].astype(str)
        print(f"✅ Pseudo-label rows: {len(tdf)}")
        return tdf[["text", "gold_label", "gold_category"]].copy()

    # 2) Optional CSV fallbacks if you have a manual train file
    for path in ["data/train.csv", "train.csv", "data/df.csv"]:
        if os.path.exists(path):
            print(f"📄 Loading training data from {path}")
            tdf = pd.read_csv(path)
            # normalize possible column variants
            if "label" in tdf.columns and "gold_label" not in tdf.columns:
                tdf = tdf.rename(columns={"label": "gold_label"})
            if "category" in tdf.columns and "gold_category" not in tdf.columns:
                tdf = tdf.rename(columns={"category": "gold_category"})
            need = {"text", "gold_label", "gold_category"}
            missing = need - set(tdf.columns)
            if missing:
                raise ValueError(f"Training CSV missing columns: {missing}")
            tdf["text"] = tdf["text"].astype(str)
            return tdf[["text", "gold_label", "gold_category"]].copy()

    # 3) Last resort: tiny demo set
    print("ℹ️ No pseudo-labels or CSV found; using small demo dataset (5 rows).")
    return pd.DataFrame({
        "text": [
            "Food is good food is great food is nice food is good",
            "Service was quick and friendly. Would return.",
            "Amazing place! Highly recommend this restaurant.",
            "Visit my site http://spam.biz – use promo code NOW!",
            "Terrible service. Will not come back."
        ],
        "gold_label": ["REJECT", "APPROVE", "APPROVE", "REJECT", "APPROVE"],
        "gold_category": ["Repetitive_Spam", "None", "Template_Spam", "ML_Detected_Spam", "None"]
    })

# Important: don't keep an old global df around
if 'df' in globals():
    del df

train_df = choose_training_df()
print(train_df.head(5))
# -----------------------

In [ ]:
from sklearn.model_selection import train_test_split

texts      = train_df['text'].astype(str).tolist()
labels     = train_df['gold_label'].tolist()
categories = train_df['gold_category'].tolist()

X_train, X_test, y_train, y_test, cat_train, cat_test = train_test_split(
    texts, labels, categories, test_size=0.3, random_state=42, stratify=labels if len(set(labels)) > 1 else None
)
detector = UnifiedSpamDetector(
    max_features=5000,
    ngram_range=(1, 3),
    spam_threshold=0.3
)

detector.fit(X_train, y_train, calibrate=True)

eval_results = evaluate_detector(detector, X_test, y_test, cat_test)
print("Unified Spam Detector Evaluation:")
eval_results




## 9. Model Persistence and Export (After Training)

In [ ]:
# Model Persistence and Training Data Export
import os
import joblib
import json
import pandas as pd
from datetime import datetime
import shutil
from pathlib import Path

print("MODEL PERSISTENCE AND DATA EXPORT")
print("=" * 60)

# Create necessary directories
os.makedirs('models/saved_models', exist_ok=True)
os.makedirs('data/training', exist_ok=True)
os.makedirs('data/predictions', exist_ok=True)
os.makedirs('results/model_info', exist_ok=True)
os.makedirs("models", exist_ok=True)

# === Save UnifiedSpamDetector (scikit-learn pipeline) ===
model_filename = f"models/saved_models/unified_spam_detector_{timestamp}.joblib"
joblib.dump(detector, model_filename)

print(f"✅ UnifiedSpamDetector saved at {model_filename}")

# Track it in model_info
model_info["sklearn_model_path"] = model_filename


# Generate timestamp for versioning
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Model information
model_info = {
    "training_timestamp": timestamp,
    "training_mode": training_mode,
    "base_model": BASE_MODEL if training_mode == "fine-tuning" else "pre-trained-only",
    "auxiliary_models": {
      "toxicity_model": TOXIC_MODEL,
      "zero_shot_model": ZERO_SHOT_MODEL
  },
    "training_data_size": len(pseudo_labels_df) if has_training_data else 0,
    "confidence_threshold": CONFIDENCE_THRESHOLDS['DEFAULT'],
    "performance_metrics": {}
}

print(f"Training Mode: {training_mode}")
print(f"Timestamp: {timestamp}")

# Save trained models
if training_mode == "fine-tuning" and TRAINED_MODELS['custom_classifier'] is not None:
    print(f"\n💾 SAVING FINE-TUNED MODELS")

    # Custom model is already saved during training
    custom_model_path = TRAINED_MODELS['custom_classifier']['model_path']
    final_model_path = f'models/saved_models/review_classifier_{timestamp}'

    # Copy to final location with timestamp
    if os.path.exists(custom_model_path):
        shutil.copytree(custom_model_path, final_model_path, dirs_exist_ok=True)
        print(f"✅ Fine-tuned model copied to: {final_model_path}")

        model_info["custom_model_path"] = final_model_path
        model_info["model_files"] = {
            "config": f"{final_model_path}/config.json",
            "model": f"{final_model_path}/pytorch_model.bin",
            "tokenizer": f"{final_model_path}/tokenizer.json"
        }

        # Test model loading
        try:
            from transformers import AutoTokenizer, AutoModelForSequenceClassification
            test_tokenizer = AutoTokenizer.from_pretrained(final_model_path)
            test_model = AutoModelForSequenceClassification.from_pretrained(final_model_path)
            print("✅ Model loading test successful")
            model_info["model_loadable"] = True
        except Exception as e:
            print(f"❌ Model loading test failed: {e}")
            model_info["model_loadable"] = False
    else:
        print(f"❌ Custom model path not found: {custom_model_path}")
        model_info["custom_model_path"] = None

else:
    print(f"\n📦 SAVING PRE-TRAINED MODEL REFERENCES")
    model_info["custom_model_path"] = None
    model_info["model_files"] = None

# Save auxiliary model pipeline state (for consistency in inference)
auxiliary_info = {
    "toxicity_model": TOXIC_MODEL,
    "zero_shot_model": ZERO_SHOT_MODEL,
    "device_used": 0 if torch.cuda.is_available() else -1,
    "models_loaded": False  # we store names; loader will instantiate
}


# Save training data if available
if has_training_data:
    print(f"\n💾 SAVING TRAINING DATA")

    # Save pseudo-labeled training data
    training_data_path = f'data/training/pseudo_labels_{timestamp}.csv'
    pseudo_labels_df.to_csv(training_data_path, index=False)
    print(f"✅ Training data saved: {training_data_path}")

    model_info["training_data_path"] = training_data_path
    model_info["training_data_size"] = len(pseudo_labels_df)

    # Save label distribution
    label_dist = pseudo_labels_df['pred_label'].value_counts().to_dict()
    model_info["training_label_distribution"] = label_dist
    print(f"   Training Labels: {label_dist}")

else:
    print(f"\n⚠️ No training data to save")
    model_info["training_data_path"] = None

# Save prediction results
if 'hf_results' in locals():
    print(f"\n💾 SAVING PREDICTION RESULTS")

    predictions_path = f'data/predictions/predictions_{timestamp}.csv'
    hf_results.to_csv(predictions_path, index=False)
    print(f"✅ Predictions saved: {predictions_path}")

    model_info["predictions_path"] = predictions_path
    model_info["predictions_count"] = len(hf_results)

    # Add performance metrics
    pred_dist = hf_results['pred_label'].value_counts().to_dict()
    avg_confidence = hf_results['confidence'].mean()

    model_info["performance_metrics"] = {
        "prediction_distribution": pred_dist,
        "average_confidence": round(float(avg_confidence), 4),
        "total_processed": len(hf_results)
    }
    print(f"   Prediction Labels: {pred_dist}")
    print(f"   Average Confidence: {avg_confidence:.4f}")

# Save comprehensive model information
model_info_path = f'results/model_info/model_info_{timestamp}.json'
with open(model_info_path, 'w') as f:
    json.dump(model_info, f, indent=2, default=str)

print(f"✅ Model info saved: {model_info_path}")

# Create deployment-ready structure for inference pipeline
print(f"\n🚀 CREATING DEPLOYMENT STRUCTURE")

# Create data/actual with sample data for inference pipeline
os.makedirs('data/actual', exist_ok=True)

# If we have results, save a sample to data/actual for the inference pipeline
if 'hf_results' in locals() and len(hf_results) > 0:
    # Create sample data for inference testing
    sample_actual = hf_results[['id', 'text']].head(3).copy()
    sample_actual.to_csv('data/actual/sample_reviews.csv', index=False)
    print("✅ Sample data for inference: data/actual/sample_reviews.csv")

# Save latest model paths for inference pipeline
latest_model_config = {
    "latest_model_info": model_info_path,
    "training_mode": training_mode,
    "custom_model_path": model_info.get("custom_model_path"),
    "auxiliary_models": auxiliary_info,
    "confidence_threshold": CONFIDENCE_THRESHOLDS['DEFAULT'],
    "timestamp": timestamp
}

with open('models/saved_models/latest_config.json', 'w') as f:
    json.dump(latest_model_config, f, indent=2)

print("✅ Latest model config: models/saved_models/latest_config.json")

# Create model loading utilities for inference
model_loader_code = '''"""
Model Loading Utilities for Review Classification Pipeline
Generated automatically during training
"""

import json
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

def load_latest_models():
    """Load the most recently trained models"""

    config_path = 'models/saved_models/latest_config.json'

    if not os.path.exists(config_path):
        raise FileNotFoundError("No trained models found. Run training pipeline first.")

    with open(config_path) as f:
        config = json.load(f)

    models = {}

    # Load custom model if available
    if config['custom_model_path'] and os.path.exists(config['custom_model_path']):
        models['custom'] = {
            'tokenizer': AutoTokenizer.from_pretrained(config['custom_model_path']),
            'model': AutoModelForSequenceClassification.from_pretrained(config['custom_model_path'])
        }

    # Load auxiliary models
    device = 0 if torch.cuda.is_available() else -1
    aux_config = config['auxiliary_models']

    models['auxiliary'] = {
    'toxicity': pipeline("text-classification",
                         model=aux_config['toxicity_model'], device=device),
    'zero_shot': pipeline("zero-shot-classification",
                          model=aux_config['zero_shot_model'], device=device),
}


    return models, config

def predict_review(text, models, config):
    """Make prediction using loaded models"""

    if 'custom' in models:
        # Use fine-tuned model
        inputs = models['custom']['tokenizer'](
            text, truncation=True, padding=True,
            max_length=256, return_tensors="pt"
        )

        with torch.no_grad():
            outputs = models['custom']['model'](**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            confidence = float(torch.max(predictions))
            predicted_class = int(torch.argmax(predictions))

        label = "REJECT" if predicted_class == 1 else "APPROVE"

    else:
        # Use zero-shot classification
        # Implementation would mirror the training notebook logic
        label = "APPROVE"  # Placeholder
        confidence = 0.5

    return {
        'label': label,
        'confidence': confidence,
        'model_type': config['training_mode']
    }
'''

Path("src/utils").mkdir(parents=True, exist_ok=True)
Path("src/__init__.py").touch(exist_ok=True)
Path("src/utils/__init__.py").touch(exist_ok=True)

with open('src/utils/model_loader.py', 'w') as f:
    f.write(model_loader_code)

print("✅ Model loader utilities: src/utils/model_loader.py")

# Summary
print(f"\n" + "="*60)
print("PERSISTENCE COMPLETE")
print("="*60)

print(f"✅ Training Mode: {training_mode}")
print(f"✅ Timestamp: {timestamp}")

if model_info.get("custom_model_path"):
    print(f"✅ Custom Model: {model_info['custom_model_path']}")
else:
    print(f"⚠️ Custom Model: None (using pre-trained)")

print(f"✅ Model Info: {model_info_path}")
print(f"✅ Latest Config: models/saved_models/latest_config.json")

if has_training_data:
    print(f"✅ Training Data: {model_info['training_data_path']}")
    print(f"   Size: {model_info['training_data_size']} examples")

if 'hf_results' in locals():
    print(f"✅ Predictions: {model_info['predictions_path']}")
    print(f"   Processed: {model_info['predictions_count']} reviews")

print(f"\n🎯 READY FOR INFERENCE PIPELINE")
print(f"   Use: models/saved_models/latest_config.json")
print(f"   Data: data/actual/sample_reviews.csv")
print(f"   Load: src/utils/model_loader.py")

print(f"\n✅ MODEL TRAINING AND PERSISTENCE COMPLETE!")

## 10. Model Performance Evaluation

In [ ]:
# --- Model Performance Evaluation with Ground Truth (robust) ---
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Optional: if you don't want seaborn, comment out the next 2 lines
import seaborn as sns
sns.set()

# 1) Choose the ground-truth dataframe you actually used to create predictions
#    Prefer train_df (pseudo-labels) if available; otherwise fall back to df.
GT_DF = None
if 'train_df' in globals():
    GT_DF = train_df
elif 'df' in globals():
    GT_DF = df

def _to_str_label(x):
    """Map numeric/boolean to APPROVE/REJECT and normalize strings."""
    if pd.isna(x):
        return np.nan
    if isinstance(x, (int, np.integer, float, np.floating)):
        # interpret 1 = REJECT, 0 = APPROVE
        return "REJECT" if int(x) == 1 else "APPROVE"
    s = str(x).strip().upper()
    if s in {"APPROVE", "REJECT"}:
        return s
    # try a few loose mappings
    if s in {"YES", "SPAM", "BLOCK"}:
        return "REJECT"
    if s in {"NO", "HAM", "ALLOW"}:
        return "APPROVE"
    return np.nan  # unknown -> will be filtered out

def evaluate_model_with_ground_truth(pred_df: pd.DataFrame, gt_df: pd.DataFrame | None):
    print("\nModel Performance Evaluation")
    print("=" * 40)

    df_eval = pred_df.copy()

    # 2) Attach ground truth if needed
    if gt_df is not None and 'gold_label' not in df_eval.columns:
        if 'id' in df_eval.columns and 'id' in gt_df.columns:
            df_eval = df_eval.merge(gt_df[['id','gold_label','gold_category']], on='id', how='left')
        else:
            print("⚠️ Could not merge ground truth by 'id' (missing column).")
    elif gt_df is None and 'gold_label' not in df_eval.columns:
        print("No ground truth available - cannot evaluate accuracy.")
        return None

    if 'gold_label' not in df_eval.columns:
        print("No ground truth column 'gold_label' found after merge.")
        return None

    # 3) Normalize labels to strings and drop NaNs
    df_eval['gold_label_norm'] = df_eval['gold_label'].apply(_to_str_label)
    df_eval['pred_label_norm'] = df_eval['pred_label'].apply(_to_str_label)

    before = len(df_eval)
    df_eval = df_eval.dropna(subset=['gold_label_norm', 'pred_label_norm'])
    after = len(df_eval)
    if after < before:
        print(f"ℹ️ Dropped {before-after} rows with missing/unknown labels after normalization.")

    # 4) Guard: ensure only APPROVE/REJECT remain
    allowed = {"APPROVE","REJECT"}
    bad_gt = set(df_eval['gold_label_norm']) - allowed
    bad_pr = set(df_eval['pred_label_norm']) - allowed
    if bad_gt or bad_pr:
        print(f"⚠️ Unexpected labels present. gold={bad_gt}, pred={bad_pr}")

    # 5) Compute metrics
    y_true = df_eval['gold_label_norm']
    y_pred = df_eval['pred_label_norm']

    acc = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred, labels=['APPROVE','REJECT'])
    cm = confusion_matrix(y_true, y_pred, labels=['APPROVE','REJECT'])

    print(f"Overall Accuracy: {acc:.3f}\n")
    print("Detailed Classification Report:")
    print(report)

    # 6) Confusion matrix plot
    try:
        plt.figure(figsize=(6,5))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=['APPROVE','REJECT'],
                    yticklabels=['APPROVE','REJECT'])
        plt.title('Confusion Matrix')
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.show()
    except Exception:
        pass

    # 7) Simple error listing
    errs = df_eval[y_true != y_pred]
    if len(errs):
        print(f"\nError Analysis ({len(errs)} mistakes / {len(df_eval)} rows)")
        print("="*50)
        for _, r in errs.iterrows():
            txt = r['text'][:120] + ("…" if isinstance(r['text'], str) and len(r['text'])>120 else "")
            print(f"ID {r.get('id','?')}: true={r['gold_label_norm']} pred={r['pred_label_norm']} | {txt}")
    else:
        print("\nNo classification errors found.")

    return {'accuracy': acc, 'n': len(df_eval), 'errors': int(len(errs))}

# ---- Run evaluation on your predictions ----
if 'hf_results' in globals():
    eval_out = evaluate_model_with_ground_truth(hf_results, GT_DF)
else:
    print("No prediction results found - run inference first.")

## 11. Pipeline Summary and Validation

In [ ]:
# Complete Pipeline Summary and Architecture Validation
print("REVIEW-RATER PIPELINE ARCHITECTURE 2.0")
print("=" * 70)

# Check if we're in Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Environment Summary
print(f"\n1. ENVIRONMENT SETUP")
print(f"   Platform: {'Google Colab' if IN_COLAB else 'Local'}")
print(f"   GPU Available: {'✅ Yes' if torch.cuda.is_available() else '❌ No'}")
print(f"   Device: {device}")

# Directory Structure Validation
print(f"\n2. DIRECTORY STRUCTURE")
expected_dirs = [
    'data/raw', 'data/clean', 'data/pseudo-label', 'data/training',
    'data/testing', 'data/actual', 'data/sample',
    'models/saved_models', 'models/cache',
    'results/predictions', 'results/inference'
]

for directory in expected_dirs:
    status = "✅" if os.path.exists(directory) else "❌"
    print(f"   {status} {directory}")

# Pipeline Architecture Summary
print(f"\n3. PIPELINE ARCHITECTURE")
print(f"   Training Flow (00_ipynb):")
print(f"      data/raw → (external) → data/clean")
print(f"      data/clean → (gemini) → data/pseudo-label")
print(f"      data/pseudo-label → data/testing + data/training")
print(f"      data/clean → data/training (combined)")
print(f"      HuggingFace training on data/training with feedback loop")
print(f"      Trained models → models/saved_models")
print(f"")
print(f"   Inference Flow (01_ipynb):")
print(f"      data/actual → models/saved_models → inference → results/inference")

# Component Status
print(f"\n4. COMPONENT STATUS")
components = {
    'Constants loaded': 'DEFAULT_MODELS' in globals(),
    'Sample data ready': 'df' in locals() or 'sample_df' in locals(),
    'HuggingFace ready': True,  # Installed in environment setup
    'Gemini available': 'gemini_available' in locals() and locals().get('gemini_available', False),
    'Directory structure': all(os.path.exists(d) for d in ['data/clean', 'data/pseudo-label', 'data/actual']),
}

for component, status in components.items():
    print(f"   {'✅' if status else '❌'} {component}")

# Model Performance Summary
print(f"\n5. MODEL PERFORMANCE")
prediction_data = None
for var_name in ['hf_results', 'all_predictions_df', 'predictions_df', 'results_df']:
    if var_name in globals():
        var_value = globals()[var_name]
        if hasattr(var_value, 'shape') and len(var_value) > 0:
            prediction_data = var_value
            break

if prediction_data is not None:
    print(f"   ✅ Predictions available: {len(prediction_data)} reviews")
    if 'confidence' in prediction_data.columns:
        avg_conf = prediction_data['confidence'].mean()
        print(f"   ✅ Average confidence: {avg_conf:.3f}")
    if 'pred_label' in prediction_data.columns:
        label_dist = prediction_data['pred_label'].value_counts()
        print(f"   ✅ Label distribution: {dict(label_dist)}")
else:
    print(f"   ❌ No prediction data available")

# Integration Readiness
print(f"\n6. INTEGRATION READINESS")
integration_checks = {
    'Structured output': prediction_data is not None,
    'Spam detection ready': True,  # Architecture supports it
    'Production deployment': os.path.exists('data/actual'),
    'Model persistence': 'save_trained_pipeline' in globals(),
    'Inference pipeline': os.path.exists('notebooks/01_inference_pipeline.ipynb'),
}

for check, status in integration_checks.items():
    print(f"   {'✅' if status else '❌'} {check}")

# Next Steps
print(f"\n7. NEXT STEPS")
print(f"   Training Phase (This Notebook):")
print(f"   1. ✅ Environment setup complete")
print(f"   2. ✅ Directory structure created")
print(f"   3. ✅ Pipeline architecture established")
print(f"   4. 🔄 Run HuggingFace pipeline (cell 8)")
print(f"   5. 🔄 Export trained models (cell 9)")
print(f"")
print(f"   Production Phase:")
print(f"   1. 📋 Place actual review data in data/actual/")
print(f"   2. 📋 Run 01_inference_pipeline.ipynb")
print(f"   3. 📋 Check results in results/inference/")

# Final Status
print(f"\n8. OVERALL STATUS")
overall_ready = all([
    os.path.exists('data/clean'),
    os.path.exists('data/actual'),
    'DEFAULT_MODELS' in globals(),
    'save_trained_pipeline' in globals()
])

if overall_ready:
    print(f"   🚀 PIPELINE READY FOR PRODUCTION")
    print(f"   ✅ Training architecture: Complete")
    print(f"   ✅ Inference architecture: Complete")
    print(f"   ✅ Data flow: Established")
    print(f"   ✅ Integration points: Ready")
else:
    print(f"   ⚠️  PIPELINE SETUP IN PROGRESS")
    print(f"   Run all cells to complete setup")

print(f"\nPIPELINE ARCHITECTURE 2.0 SUMMARY")
print(f"=" * 70)
print(f"✅ data/raw → data/clean → data/pseudo-label → data/training/testing")
print(f"✅ HuggingFace training with Gemini feedback loop")
print(f"✅ models/saved_models for production deployment")
print(f"✅ data/actual → 01_ipynb → results/inference")
print(f"✅ Spam detection integration ready")
print(f"✅ Complete separation of training and inference phases")